In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from stability.data import random_split
from stability.bootstrap import bootstrap_indices

First, we need to list all the tiles. We also read in the `y.csv` files, because they have a column describing which subject each patch came from (we could do string manipulation on the paths, but having an index file is more reliable).

In [ ]:
data_dir = Path(os.environ["DATA_DIR"])
paths = {
    "tile": Path(data_dir).glob("**/*npy"),
    "y": Path(data_dir).glob("**/y*csv")
}

y = []
for p in paths["y"]:
    y.append(pd.read_csv(p))

Now, we can generate the split across participants and link the assignments back to the tiles associated with them. The resulting splits are saved in `Xy.csv`.

In [ ]:
y_df = pd.concat(y)
subjects = y_df["i"].unique()

split = random_split(subjects, [0.7, 0.1, 0.2])
split.columns = ["i", "split"]

x_col = [col for col in y_df if col.startswith("X")]
split = pd.merge(y_df[["i", "w", "h", "y"] + x_col], split)
split["path"] = split.apply(lambda r: f"tiles/{r.i}_{str(r.w)}-{str(r.h)}.npy", axis=1)
split.to_csv(data_dir / "Xy.csv", index=False)

For reference, some example rows from this file are,

In [ ]:
split[:3]

In [ ]:
b = bootstrap_indices(len(split.loc[split["split"] == "train", "path"]), 20, data_dir / f"bootstraps.csv")